# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Insert consolidated csv file to Apache Casssandra database

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS cassandra 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('cassandra')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Based on the three questions above, I need to create three tables. I will setup sessionid and iteminsession as primary key in first table, userid, sessionid and iteminsession as primary key in the second table, and song, artist, userid and seesionid as primary key is the third table.

In [8]:
# Drop three tables if exists to avoild throwing out errors
song_info_session_drop = "DROP TABLE IF EXISTS song_info_session"
song_playlist_session_drop = "DROP TABLE IF EXISTS song_playlist_session"
user_info_songname_drop = "DROP TABLE IF EXISTS user_info_songname"
drop_table_queries = [song_info_session_drop, song_playlist_session_drop, user_info_songname_drop]

for query in drop_table_queries:
    session.execute(query)

In [9]:
create_song_info_session = "CREATE TABLE IF NOT EXISTS song_info_session \
                            (sessionid int, iteminsession int, artist text, firstname text, gender text, lastname text, \
                            length float, level text, location text, song text, userid int, \
                            PRIMARY KEY (sessionid, iteminsession))"

create_song_playlist_session = "CREATE TABLE IF NOT EXISTS song_playlist_session (userid int, sessionid int, iteminsession int, \
                                artist text, firstname text, gender text, lastname text, length float, level text, \
                                location text, song text, PRIMARY KEY ((userid, sessionid), iteminsession))"

create_user_info_songname = "CREATE TABLE IF NOT EXISTS user_info_songname (song text, userid int, artist text, \
                            firstname text, gender text, iteminsession int, lastname text, length float,\
                            level text, location text, sessionid int, PRIMARY KEY (song, userid))"

create_table_queries = [create_song_info_session, create_song_playlist_session, create_user_info_songname]

try:
    for query in create_table_queries:
        session.execute(query)
except Exception as e:
    print(e)

### Read consolidated csv file and insert into three tables and then create a consolidated csv file.

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    table_list = ["song_info_session", "song_playlist_session", "user_info_songname"]
    for line in csvreader:
        for table in table_list:
            if table == "song_info_session":
                query = "INSERT INTO " + table + "(sessionid, iteminsession, artist, firstname, gender, lastname, \
                    length, level, location, song, userid) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

                session.execute(query, (int(line[8]), int(line[3]), line[0], line[1], line[2], line[4], float(line[5]),\
                                        line[6], line[7], line[9], int(line[10])))
            elif table == "song_playlist_session":
                query = "INSERT INTO " + table + "(userid, sessionid, iteminsession, artist, firstname, gender, lastname, \
                    length, level, location, song) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

                session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[2], line[4],\
                                            float(line[5]), line[6], line[7], line[9]))
            else:
                query = "INSERT INTO " + table + "(song, userid, artist, firstname, gender, iteminsession, lastname, \
                    length, level, location, sessionid) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

                session.execute(query, (line[9], int(line[10]), line[0], line[1], line[2], int(line[3]), line[4], \
                                        float(line[5]), line[6], line[7], int(line[8])))
                

#### Now let's answer the first question.

In [16]:
# Question 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
answer1_query = "SELECT artist, song, length FROM song_info_session WHERE sessionid = 338 and iteminsession = 4"
try:
    rows = session.execute(answer1_query)
except Exception as e:
    print(e)
    
for row in rows:
    print ("Artist: ", row.artist,"Song: ", row.song,"Length: ", row.length)

Artist:  Faithless Song:  Music Matters (Mark Knight Dub) Length:  495.30731201171875


#### Based on the result, only one result is returned.
| Artust        | Song Title                      | Length             |
| :------------ |:--------------------------------| :------------------|
|Faithless      | Music Matters (Mark Knight Dub) | 495.30731201171875 |

### Let's go to the second question.

In [17]:
## Question 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

answer2_query = "select artist, song, firstname, lastname from song_playlist_session WHERE userid = 10 and sessionid = 182"
try:
    rows = session.execute(answer2_query)
except Exception as e:
    print(e)
    
for row in rows:
    print ("Artist: ", row.artist,"Song Title: ", row.song, "FN: ", row.firstname,"LN: ", row.lastname)
                    

Artist:  Down To The Bone Song Title:  Keep On Keepin' On FN:  Sylvie LN:  Cruz
Artist:  Three Drives Song Title:  Greece 2000 FN:  Sylvie LN:  Cruz
Artist:  Sebastien Tellier Song Title:  Kilometer FN:  Sylvie LN:  Cruz
Artist:  Lonnie Gordon Song Title:  Catch You Baby (Steve Pitron & Max Sanna Radio Edit) FN:  Sylvie LN:  Cruz


#### Based on the result, four records have been returned.
| Artust          | Song Title                                           | First Name| Last Name|
| :---------------|:-----------------------------------------------------| :---------|:---------|
|Down To The Bone | Keep On Keepin' On                                   | Sylvie    | Cruz     |
|Three Drives     | Greece 2000                                          | Sylvie    | Cruz     |
|Sebastien Tellier| Kilometer                                            | Sylvie    | Cruz     |
|Lonnie Gordon    | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie    | Cruz     |

### Here comes to question 3:

In [18]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
answer3_query = "select firstname, lastname from user_info_songname WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(answer3_query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


#### Based on the result, three records have been returned.
| First Name| Last Name|
| :---------|:---------|
| Jacqueline| Lynch    |
| Tegan     | Levine   |
| Sara      | Johnson  |

### Drop the tables before closing out the sessions

In [19]:
song_info_session_drop = "DROP TABLE IF EXISTS song_info_session"
song_playlist_drop = "DROP TABLE IF EXISTS song_playlist_session"
user_info_songname_drop = "DROP TABLE IF EXISTS user_info_songname"
drop_table_queries = [song_info_session_drop, song_playlist_drop, user_info_songname_drop]

for query in drop_table_queries:
    session.execute(query)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()